In [1]:
import re
import networkx as nx
import matplotlib.pyplot as plt
import pydot
import matplotlib.cm as cm
import matplotlib.colors as mcolors
import pandas as pd
import subprocess

from copy import deepcopy

In [ ]:
def parse_alb(alb_file_name):
    """Reads assembly line balancing instance .alb file, returns dictionary with the information"""
    parse_dict = {}
    alb_file = open(alb_file_name).read()
    # Get number of tasks
    num_tasks = re.search("<number of tasks>\n(\\d*)", alb_file)
    parse_dict["num_tasks"] = int(num_tasks.group(1))

    # Get cycle time
    cycle_time = re.search("<cycle time>\n(\\d*)", alb_file)
    parse_dict["cycle_time"] = int(cycle_time.group(1))

    # Order Strength
    order_strength = re.search("<order strength>\n(\\d*,\\d*)", alb_file)
    
    if order_strength:
        parse_dict["original_order_strength"] = float(order_strength.group(1).replace(",", "."))
    else:
        order_strength = re.search("<order strength>\n(\\d*.\\d*)", alb_file)
        parse_dict["original_order_strength"] = float(order_strength.group(1))

    # Task_times
    task_times = re.search("<task times>(.|\n)+?<", alb_file)

    # Get lines in this regex ignoring the first and last 2
    task_times = task_times.group(0).split("\n")[1:-2]
    task_times = {task.split()[0]: int(task.split()[1]) for task in task_times}
    parse_dict["task_times"] = task_times

    # Precedence relations
    precedence_relations = re.search("<precedence relations>(.|\n)+?<", alb_file)
    precedence_relations = precedence_relations.group(0).split("\n")[1:-2]
    precedence_relations = [task.split(",") for task in precedence_relations]
    parse_dict["precedence_relations"] = precedence_relations
    return parse_dict

def write_to_alb(salbp_dict, alb_file_name):
    """Writes the SALBP dictionary to an .alb file"""
    #Format of alb:
    # <number of tasks>
    # no_tasks
    # <cycle time>
    # cycle_time
    #<task times>
    #task_id task_time
    #<precedence relations>
    #task_id,task_id


    # Write number of tasks
    alb = "<number of tasks>\n"
    alb += str(salbp_dict["num_tasks"]) + "\n"
    # Write cycle time
    alb += "<cycle time>\n"
    alb += str(salbp_dict["cycle_time"]) + "\n"
    # Write task times
    alb += "<task times>\n"
    for task_id, task_time in salbp_dict["task_times"].items():
        alb += task_id + " " + str(task_time) + "\n"
    # Write precedence relations
    alb += "<precedence relations>\n"
    for relation in salbp_dict["precedence_relations"]:
        alb += relation[0] + "," + relation[1] + "\n"
    #ends the file
    alb += "<end>"
    with open(alb_file_name, "w") as alb_file:
        alb_file.write(alb)
    


SALBP_dict = parse_alb("/Users/letshopethisworks2/Documents/phd_paper_material/MALBP_instance_generation/SALBP_benchmark/small data set_n=20/instance_n=20_1.alb")
write_to_alb(SALBP_dict, "test.alb")

In [ ]:
my_test_alb = {'num_tasks': 12,
 'cycle_time': 10,
 'original_order_strength': 0.268,
 'task_times': {'1': 3,
  '2': 8,
  '3': 3,
  '4': 2,
  '5': 4,
  '6': 2,
  '7': 5,
  '8': 1,
  '9': 9,
  '10': 4,
  '11': 6,
  '12': 2,
    },
 'precedence_relations': [
     ['1', '5'],
  ['2', '5'],
  ['3', '5'],
  ['3', '4'],
  ['4', '9'],
  ['5', '6'],
  ['5', '7'],
  ['5', '8'],
  ['6', '11'],
  ['7', '11'],
  ['8', '11'],
  ['9', '10'],
  ['10', '12'],
  ['11', '12']
  ]
  }

In [ ]:
SALBP_dict

In [ ]:
! ls BBR-for-SALBP1

In [ ]:
! ../BBR-for-SALBP1/SALB/SALB/salb  -p 2 "test.alb" 

In [ ]:


def random_task_time_change(SALBP_dict, multiplier = 1.5):
    """Increases a random task time by 1"""
    import random
    task_id = random.choice(list(SALBP_dict["task_times"].keys()))
    SALBP_dict["task_times"][task_id] *= multiplier
    return SALBP_dict

def task_time_change(SALBP_dict, task_id, multiplier = 1.5, debug = False):
    """Increases a random task time by 1"""
    if debug:
        print("Changing task", task_id, "time by", multiplier)
    SALBP_dict["task_times"][task_id] *= multiplier
    return SALBP_dict

def precedence_removal(SALBP_dict, edge_index):
    """Removes a precedence relation"""
    SALBP_dict["precedence_relations"].pop(edge_index)
    return SALBP_dict
    

def parse_bb_salb1_out(text):
    '''gets the number of stations, optimal flag and cpu time from the output of the salb1 program'''
    output = text.stdout.decode("utf-8")
    # Regular expression to capture the required values
    match = re.search(r"verified_optimality\s*=\s*(\d+);\s*value\s*=\s*(\d+);\s*cpu\s*=\s*([\d.]+)", output)

    if match:
        verified_optimality = int(match.group(1))
        value = int(match.group(2))
        cpu = float(match.group(3))

    else:
        print("Pattern not found.")
    return value, verified_optimality, cpu

def generate_results(fp = "/Users/letshopethisworks2/Documents/phd_paper_material/MALBP_instance_generation/SALBP_benchmark/small data set_n=20/" ,  instance_name = "instance_n=20_", ext = ".alb", start=1, stop = 300):
    results = []
    for i in range(start,stop):
        SALBP_dict_orig = parse_alb(f"{fp}{instance_name}{i}{ext}")
        bin_dict = deepcopy(SALBP_dict_orig)
        print("Running instance: ", i)
        for j in range(len(SALBP_dict_orig["precedence_relations"])):
            SALBP_dict = deepcopy(SALBP_dict_orig)
            SALBP_dict =precedence_removal(SALBP_dict, j)
            write_to_alb(SALBP_dict, "test.alb")
            output = subprocess.run([ex_fp, "test.alb"], stdout=subprocess.PIPE)
            no_stations, optimal, cpu = parse_bb_salb1_out(output)
            result = {"instance:": f"instance_n=20_{i}", "precedence_relation": j, "no_stations": no_stations, "optimal": optimal, "cpu": cpu}
            save_backup(backup_name, result)
            results.append(result)

        #calculates bin packing lower bound
        bin_dict['precedence_relations'] = []
        write_to_alb(bin_dict, "test.alb")
        output = subprocess.run([ex_fp, "test.alb"], stdout=subprocess.PIPE)
        no_stations, optimal, cpu = parse_bb_salb1_out(output)
        result = {"instance:": f"instance_n=20_{i}", "precedence_relation": "None", "no_stations": no_stations, "optimal": optimal, "cpu": cpu}
        save_backup(backup_name, result)
            
        results.append(result)
    return results

#reads the results csv
#results_df = pd.read_csv("task_20_bin_lb.csv")
#results_df = pd.DataFrame(results)
#saves the results df to a csv file
#results_df.to_csv("tasks20_test.csv")
# results = generate_results(start=400, stop = 525)
# results_df = pd.DataFrame(results)
# results_df.to_csv("tasks20_3.csv")

In [ ]:
res_1 = pd.read_csv("task_20_bin_lb.csv")
res_2 = pd.read_csv("tasks20_2.csv")
res_3 = pd.read_csv("tasks20_3.csv")

results_df = pd.concat([res_1, res_2, res_3])

In [ ]:
results_df

In [ ]:
lb_df = results_df[results_df["precedence_relation"].isna() == True].copy()
#removes the rows with None precedence relations
results_df = results_df[results_df['precedence_relation'].isna() == False]

lb_df

In [ ]:
results_df.head(20)

In [ ]:
#gets the min and max number of stations for each instance
min_and_max = results_df.groupby("instance:")["no_stations"].agg(["min", "max"])
min_and_max.reset_index(inplace = True)
#adds in lb values
lb_df['bin_lb'] = lb_df['no_stations']
min_and_max = pd.merge(min_and_max, lb_df[["instance:", "bin_lb"]], on = "instance:")
min_and_max

In [ ]:
#counts the number of times min does not equal max
min_and_max["min_not_equal_max"] = min_and_max["min"] != min_and_max["max"]
min_and_max["min_not_equal_max"].sum()

In [ ]:
#counts the number of time the bin_lb is less than the min
min_and_max["bin_lb_less_than_min"] = min_and_max["bin_lb"] < min_and_max["min"]
min_and_max["bin_lb_less_than_min"].sum()

In [ ]:
#counts the number of time the bin_lb is less than the max
min_and_max["bin_lb_less_than_max"] = min_and_max["bin_lb"] < min_and_max["max"]
print("bin lb dif", min_and_max["bin_lb_less_than_max"].sum())
#filters for the instances where the bin_lb is les than the max
min_and_max[min_and_max["bin_lb_less_than_max"] == True]

In [ ]:
#prints the instances where min does not equal max
interesting_instances = min_and_max[min_and_max["min_not_equal_max"]]
interesting_instances['instance:'].nunique()

In [ ]:
inst_20_101 = results_df[results_df["instance:"] == "instance_n=20_101"]
inst_20_101

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import pydot
import matplotlib.cm as cm
import matplotlib.colors as mcolors

def plot_salbp_graph(SALBP_dict):
    G = nx.DiGraph()
    G.add_nodes_from(SALBP_dict["task_times"].keys())
    G.add_edges_from(SALBP_dict["precedence_relations"])
    #prints the edges
    print("from dict", SALBP_dict["precedence_relations"])
    #prints the edges from the graph
    print("from graph", G.edges())
    nx.draw(G, with_labels = True)
    plt.show()

def plot_salbp_edge_removal_graph(SALBP_dict, instance_name, res_df):
    '''Colors the edges by the number of stations in res_df'''
    G = nx.DiGraph()
    G.add_nodes_from(SALBP_dict["task_times"].keys())
    G.add_edges_from(SALBP_dict["precedence_relations"])
    edge_colors = []
    for edge in G.edges():
        edge_index = SALBP_dict["precedence_relations"].index(list(edge))
        no_stations = res_df[(res_df["instance:"] == instance_name) & (res_df["precedence_relation"] == edge_index)]["no_stations"].values[0]
        edge_colors.append(no_stations)
    #saves edge colors as graph attribute
    nx.set_edge_attributes(G, dict(zip(G.edges(), edge_colors)), "value")
    pos = nx.nx_pydot.graphviz_layout(G, prog = "dot")
   # Define colormap
    unique_values = list(set(edge_colors))
    print(unique_values)
    color_map = cm.get_cmap('viridis', len(unique_values))
    print("color map", color_map)
    cmap = mcolors.ListedColormap([color_map(val) for val in unique_values])

    # Draw graph
    #creates ax
    fig, ax = plt.subplots()
    edges = nx.draw_networkx_edges(G, pos, edge_color=edge_colors, edge_cmap=cmap, edge_vmin=min(edge_colors), edge_vmax=max(edge_colors), ax=ax)
    nx.draw_networkx_nodes(G, pos, ax=ax)
    nx.draw_networkx_labels(G, pos, ax=ax)

    # Add colorbar
    handles = [plt.Line2D([0], [0], marker='o', color = color_map(val), label=val, markersize=10) for val in unique_values]
    plt.legend(handles=handles, loc="best")

    plt.show()
    return G


def draw_graph_with_discrete_legend(SALBP_dict, res_df, instance_name,  ax=None):
    G = nx.DiGraph()
    G.add_nodes_from(SALBP_dict["task_times"].keys())
    G.add_edges_from(SALBP_dict["precedence_relations"])

    edge_colors = []
    edge_values = []  # Store unique edge values for legend

    for edge in G.edges():
        edge_index = SALBP_dict["precedence_relations"].index(list(edge))
        no_stations = res_df[(res_df["instance:"] == instance_name) & 
                             (res_df["precedence_relation"] == edge_index)]["no_stations"].values[0]
        edge_colors.append(no_stations)
        if no_stations not in edge_values:
            edge_values.append(no_stations)

    # Save edge colors as graph attribute
    nx.set_edge_attributes(G, dict(zip(G.edges(), edge_colors)), "value")

    # Graph layout
    pos = nx.nx_pydot.graphviz_layout(G, prog="dot")

    # Define discrete colormap
    unique_values = sorted(edge_values)
    num_colors = len(unique_values)
    cmap = plt.cm.get_cmap("Set1", num_colors)  # Use a qualitative colormap
    color_map = {val: cmap(i) for i, val in enumerate(unique_values)}  # Assign colors to unique values

    # Assign discrete colors to edges
    edge_color_list = [color_map[val] for val in edge_colors]

    # Draw graph
    if ax is None:
        fig, ax = plt.subplots()
    edges = nx.draw_networkx_edges(G, pos, edge_color=edge_color_list, ax=ax)
    nx.draw_networkx_nodes(G, pos, ax=ax)
    nx.draw_networkx_labels(G, pos, ax=ax)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['left'].set_visible(False)
    # Create legend
    handles = [plt.Line2D([0], [0], color=color_map[val], lw=2, label=f"No. of Stations: {val}") for val in unique_values]
    #ax.legend(handles=handles, loc="best")


    return G

i = 1

test_salb = parse_alb(f"/Users/letshopethisworks2/Documents/phd_paper_material/MMABPWW/SALBP_benchmark/small data set_n=20/instance_n=20_{i}.alb")
#test_g = plot_salbp_edge_removal_graph(test_salb, f"instance_n=20_{i}", results_df)
test_g = draw_graph_with_discrete_legend(test_salb, results_df, f"instance_n=20_{i}")
#saves graph to a gephi readable file
nx.write_gexf(test_g, "test_salb.gexf")
#plot_salbp_graph(test_salb)

In [ ]:
interesting_instances['instance:'].values

In [ ]:
#creates a plot of the 27 graphs of interest
fig, axs = plt.subplots(5, 11, figsize=(20, 20))
axs = axs.ravel()
for idx, i in enumerate(interesting_instances['instance:'].values):
    test_salb = parse_alb(f"/Users/letshopethisworks2/Documents/phd_paper_material/MALBP_instance_generation/SALBP_benchmark/small data set_n=20/{i}.alb")
    #test_g = plot_salbp_edge_removal_graph(test_salb, f"instance_n=20_{i}", results_df)
    test_g = draw_graph_with_discrete_legend(test_salb, results_df, i, ax=axs[idx])
    #adds test_g to the axs

plt.show()

In [ ]:
# results_50 = pd.read_csv("SALBP_edge_solutions.csv")
# #changes 20 in instance: to 50
# results_50['instance:'] = results_50['instance:'].str.replace("20", "50")
# #creates a seperate dataframe for the instances with None precedence relations
# lb_df_50 = results_50[results_50["precedence_relation"].isna() == True].copy()
# #removes the rows with None precedence relations
# results_50 = results_50[results_50['precedence_relation'].isna() == False]
# #gets the min and max number of stations for each instance
# min_and_max_50 = results_50.groupby("instance:")["no_stations"].agg(["min", "max"])
# min_and_max_50.reset_index(inplace = True)
# #adds in lb values
# lb_df_50['bin_lb'] = lb_df_50['no_stations']
# min_and_max_50 = pd.merge(min_and_max_50, lb_df_50[["instance:", "bin_lb"]], on = "instance:")
# #counts the number of times min does not equal max
# min_and_max_50["min_not_equal_max"] = min_and_max_50["min"] != min_and_max_50["max"]
# min_and_max_50["min_not_equal_max"].sum()

In [ ]:
#looks at instances where the min is not equal to the max
# interesting_instances_50 = min_and_max_50[min_and_max_50["min_not_equal_max"]]
# interesting_instances_50

In [ ]:
results_df
#merges min and max 

In [ ]:
#merges results with min and max
results_df = pd.merge(results_df, min_and_max, on = "instance:")
results_df

In [ ]:
tasks_50_1 = pd.read_csv("med_50/instances_50_1_through_331.csv")
tasks_50_2 = pd.read_csv("med_50/instances_50_331through399.csv")
tasks_50_3 = pd.read_csv("med_50/instances_50_400through524.csv")
tasks_50 = pd.concat([ tasks_50_1, tasks_50_2, tasks_50_3])
tasks_50.head()

#reads the xlsx file
task_50_details = pd.read_excel("med_50/Details of the medium data set (n=50 and n=50permuted).xlsx")
task_50_details.head()

#first row is the columns
task_50_details.columns = task_50_details.iloc[0]
#removes the first row
task_50_details = task_50_details.iloc[1:]
task_50_details.head()


#removes <> and whitespace from the columns
task_50_details.columns = task_50_details.columns.str.replace("<", "").str.replace(">", "").str.strip()
task_50_details.columns
#left merges on instance: and Filename columns
tasks_50 = pd.merge(tasks_50, task_50_details, left_on = "instance:", right_on = "Filename")
tasks_50.head()
#drops the Filename column and Unnamed: 0
tasks_50.drop(columns = ["Filename", "Unnamed: 0"], inplace = True)
tasks_50.head()


#puts the rows with no_stations as none in a seperate dataframe
lb_df_50 = tasks_50[tasks_50["precedence_relation"].isna() == True].copy()
#removes the rows with None precedence relations
tasks_50 = tasks_50[tasks_50['precedence_relation'].isna() == False]
#gets the min and max number of stations for each instance
min_and_max_50 = tasks_50.groupby("instance:")["no_stations"].agg(["min", "max"])
min_and_max_50.reset_index(inplace = True)
# #renames the min and max columns as lowest_cost and original_optimal
min_and_max_50.rename(columns = {"min": "lowest_cost", "max": "original_optimal"}, inplace = True)
# #merges min and max with the tasks_50 dataframe
lb_df_50['bin_lb'] = lb_df_50['no_stations']
min_and_max_50 = pd.merge(min_and_max_50, lb_df_50[["instance:", "bin_lb"]], on = "instance:")
tasks_50 = pd.merge(tasks_50, min_and_max_50, on = "instance:")
# #adds in lb values

In [ ]:
#gets the instances where min and max have a gap of more than 1
bad_eggs = min_and_max_50[min_and_max_50["original_optimal"] - min_and_max_50["lowest_cost"] > 1]
bad_eggs

In [ ]:
min_and_max_50[min_and_max_50["instance:"] == "instance_n=50_31"	]

In [ ]:
#creates bins for order strength 0-0.2, 0.2-0.4, 0.4-0.6, 0.6-0.8, 0.8-1
tasks_50['order_strength_bins'] = pd.cut(tasks_50['Order strength'], bins = [0, 0.2, 0.4, 0.6, 0.8, 1], labels = ["0-0.2", "0.2-0.4", "0.4-0.6", "0.6-0.8", "0.8-1"])
tasks_50['min_not_equal_max'] = tasks_50["lowest_cost"] != tasks_50["original_optimal"]
tasks_50['bin_lb_less_than_min'] = tasks_50["bin_lb"] < tasks_50["lowest_cost"]
tasks_50['bin_lb_less_than_max'] = tasks_50["bin_lb"] < tasks_50["original_optimal"]
tasks_50['bin_lb_less_than_max'].sum()

In [ ]:
intereseting_50 = tasks_50[tasks_50["min_not_equal_max"] == True]
boring_50 = tasks_50[tasks_50["min_not_equal_max"] == False]
print("interesting", intereseting_50['instance:'].nunique(), "boring", boring_50['instance:'].nunique())

In [ ]:
def longest_chain_containing_node(G, node):
    if node not in G:
        return 0

    # Step 1: Get longest path to `node`
    longest_to_node = {n: 0 for n in G}  # Dictionary to store longest path to each node
    for n in nx.topological_sort(G):  # Process in topological order
        for pred in G.predecessors(n):
            longest_to_node[n] = max(longest_to_node[n], longest_to_node[pred] + 1)

    # Step 2: Get longest path from `node`
    longest_from_node = {n: 0 for n in G}  # Dictionary to store longest path from each node
    for n in reversed(list(nx.topological_sort(G))):  # Process in reverse topological order
        for succ in G.successors(n):
            longest_from_node[n] = max(longest_from_node[n], longest_from_node[succ] + 1)

    # Step 3: Compute total longest chain containing the node
    return longest_to_node[node] + longest_from_node[node] + 1  # +1 to include the node itself


def get_longest_chains_edges(G):
    # Step 1: Get longest path to `node`
    longest_to_node = {n: 0 for n in G}  # Dictionary to store longest path to each node
    for n in nx.topological_sort(G):  # Process in topological order
        for pred in G.predecessors(n):
            longest_to_node[n] = max(longest_to_node[n], longest_to_node[pred] + 1)

    # Step 2: Get longest path from `node`
    longest_from_node = {n: 0 for n in G}  # Dictionary to store longest path from each node
    for n in reversed(list(nx.topological_sort(G))):  # Process in reverse topological order
        for succ in G.successors(n):
            longest_from_node[n] = max(longest_from_node[n], longest_from_node[succ] + 1)
    edge_list = []
    # Step 3: Compute total longest chain containing the edge
    for edge in G.edges():
        edge_list.append((edge, longest_to_node[edge[0]] + longest_from_node[edge[1]] + 1))
    return edge_list

In [ ]:
import random
G = nx.DiGraph()
G.add_edges_from([(0, 1), (0, 2), (1, 3), (2, 3), (3, 4), (5,6), (2,6)] )
#adds node attributes "weight to the nodes"

for node in G.nodes():
    G.nodes[node]["weight"] = random.randint(1, 10)

for n in nx.topological_sort(G):
    print(n)

#draws the graph
nx.draw(G, with_labels = True)
plt.show()

In [ ]:
G.nodes(data=True)

In [ ]:
import numpy as np

def get_all_positional_weight(G):
    '''Gets the positional weight of the graph'''
    positional_weight = {}
    trans_G = nx.transitive_closure(G)
    #positional weight is the weight of the node plus the weight of its children
    for node in trans_G.nodes():
        positional_weight[node] = trans_G.nodes[node]["weight"]
        for child in trans_G.neighbors(node):
            positional_weight[node] += trans_G.nodes[child]["weight"]
    return positional_weight

def get_all_reverse_positional_weight(G):
    '''Gets the reverse positional weight of the graph'''
    rev_G = G.reverse()
    rpw = get_all_positional_weight(rev_G)
    return rpw

def get_all_children(G):
    '''Gets all the children of the nodes in the  graph'''
    children_dict = {}
    for node in G.nodes():
        children_dict[node] = list(G.successors(node))
    return children_dict

def get_all_parents(G):
    '''Gets all the parents of the nodes in the graph'''
    parents_dict = {}
    for node in G.nodes():
        parents_dict[node] = list(G.predecessors(node))
    return parents_dict

def get_all_succesors(G):
    '''Gets all the succesors of the nodes in the graph'''
    trans_G = nx.transitive_closure(G)
    succesors_dict = {}
    for node in trans_G.nodes():
        succesors_dict[node] = list(G.predecessors(node))
    return succesors_dict

def get_edge_neighbor_max_min_avg_std(G):
    '''For each edge, gets the maximum and minimum weight of its neighbors'''
    edge_neighbor_max_min = {}
    for edge in G.edges():
        #gets the weights of the predecessors of the first node in the edge
        pred_weights = [G.nodes[pred]["weight"] for pred in G.predecessors(edge[0])] 
        print("pred weights", pred_weights)
        #gets the weights of the successors of the second node in the edge
        succ_weights = [G.nodes[succ]["weight"] for succ in G.successors(edge[1])] 
        print("succ weights", succ_weights)
        #adds the max and min of the weights to the edge_neighbor_max_min dictionary
        weights = pred_weights + succ_weights
        if weights:
            edge_neighbor_max_min[edge] = {"max": max(weights), "min": min(weights), "avg": sum(weights)/len(weights), "std": np.std(weights)}
        else:
            edge_neighbor_max_min[edge] = {"max": 0, "min": 0, "avg": 0, "std": 0}
    return edge_neighbor_max_min





In [ ]:
get_edge_neighbor_max_min_avg_std(G)

In [ ]:
tasks_50

In [ ]:
tasks_50.columns

In [ ]:
#gets all of the instances where optimal is not equal to original optimal
interesting_instances_50 = tasks_50[tasks_50["Upper bound on the number of stations"] != tasks_50["original_optimal"]]
interesting_instances_50['instance:'].nunique()

In [ ]:
interesting_instances_50.columns

In [ ]:
interesting_instances_50[['No of stations in optimum', 'original_optimal', 'Optimum found? -- 1 for "Yes"','Upper bound on the number of stations', 'bin_lb', "lowest_cost"]].tail()


In [ ]:
interesting_instances_50[interesting_instances_50['No of stations in optimum'].isin([12,13,34]) ]